In [1]:
from pprint import pprint
import os
import sys

your_cluster_name = "mustafa5"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## If delete cfncluster after job is done./
delete_cfncluster= False

print("variables set")

variables set


In [2]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import AddonsManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa5 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="52.43.127.225"
Output:"MasterPrivateIP"="172.31.27.132"
Output:"GangliaPublicURL"="http://52.43.127.225/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.27.132/ganglia/"

connecting
connected


### Check Scripts on Cluster

In [3]:
scripts = AddonsManager.get_scripts_dict(ssh_client)
print()
print("Supported Pipelines:", AddonsManager.get_all_pipeline_names(scripts))

Executing python /shared/workspace/Pipelines/util/GetScripts.py

Supported Pipelines: ['RNASeq', 'ChiPSeq', 'DNASeq', 'SmallRNASeq']


In [4]:
##Check which workflows are in a given pipeline##

#can be set to a supported pipeline name or "all"
target_pipeline = "DNASeq"

print("Supported Workflows in {} Pipeline(s): ".format(target_pipeline), end="")
pprint(AddonsManager.get_workflows_in_pipeline(scripts, target_pipeline), indent=2)

Supported Workflows in DNASeq Pipeline(s): ['bwa_gatk', 'bwa_mutect']


In [5]:
#can be set to a supported pipeline or "all"
target_pipeline = "DNASeq"

#can be set to a support workflow or "all"
target_workflow = "all"

pprint(AddonsManager.get_scripts(scripts, target_pipeline, target_workflow))

{'All Pipelines': ['fastqc.sh', 'multiqc.sh', 'trim.sh', 'run.sh'],
 'All Workflows': ['merge.sh',
                   'split.sh',
                   'sort.sh',
                   'bwa.sh',
                   'post.sh',
                   'dedup.sh'],
 'bwa_gatk': ['haplo.sh', 'group_vcf.sh', 'filter.sh'],
 'bwa_mutect': ['pair_merge.sh', 'mutect.sh']}


In [6]:
##Find what steps call a script##
print(AddonsManager.get_steps_calling_script(ssh_client, scripts, "merge.sh"))

Executing cat /shared/workspace/Pipelines/config/tools.yaml
merge.sh called from:
merge in all Workflows in DNASeq
bam_merge in all Workflows in DNASeq



In [10]:
##Print out a script##

#all targets must be set to a valid pipeline/workflow/script
#use cells above to check valid options
target_pipeline = "DNASeq"
target_workflow = "bwa_mutect"
target_script = "mutect.sh"


loc, file_cat = AddonsManager.cat_script(ssh_client, scripts, target_pipeline, target_workflow, target_script)
print(loc)
print()
print(file_cat)

Executing cat /shared/workspace/Pipelines/scripts/DNASeq/bwa_mutect/mutect.sh
Workflow Specific

#!/bin/bash

project_name=$1
workflow=$2
file_suffix=$3  #extension of input file, does not include .gz if present in input
root_dir=$4
normal_sample=$5
tumor_sample=$6
input_address=$7    #this is an s3 address e.g. s3://path/to/input/directory
output_address=$8   #this is an s3 address e.g. s3://path/to/output/directory
log_dir=$9
is_zipped=${10}    #either "True" or "False", indicates whether input is gzipped
num_threads=${11}
chromosome=${12}
##TUMOR VS NORMAL


#logging
log_dir=$log_dir/$normal_sample
mkdir -p $log_dir
log_file=$log_dir/"mutect.$chromosome.log"
exec 1>>$log_file
exec 2>>$log_file

status_file=$log_dir/'status.log'
touch $status_file

#prepare output directories
workspace=$root_dir/$project_name/$workflow/$normal_sample
mkdir -p $workspace
mkdir -p $workspace/temp

echo "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
date
echo "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"

check_step_a

### Check Software on Cluster

In [11]:
software = AddonsManager.get_software_dict(ssh_client)
print("All Installed Software: ")
pprint(software)

Executing python /shared/workspace/Pipelines/util/GetSoftware.py
All Installed Software: 
{'FastQC': ['0.11.3'],
 'HTSeq': ['0.9.1 (installed with pip)'],
 'RSEM': ['1.3.0'],
 'STAR': ['2.3.0e', '2.5.1a', '2.5.3a'],
 'Trimmomatic': ['0.36'],
 'bedtools2': ['2.19.1'],
 'blat': ['36x1'],
 'bowtie': ['1.0.1'],
 'bowtie2': ['2.3.2-legacy'],
 'bwa': ['0.7.12-r1039'],
 'cutadapt': ['1.14 (installed with conda)'],
 'gatk': ['3.8-0'],
 'homer': ['4.8.3'],
 'java': ['jre1.8.0_144'],
 'kallisto': ['0.43.1'],
 'multiqc': ['1.3 (installed with pip)'],
 'picard': ['1.96'],
 'sambamba': ['0.4.7'],
 'samblaster': ['0.1.21'],
 'samtools': ['1.1'],
 'tabix': ['0.2.6'],
 'vcftools': ['0.1.12b'],
 'weblogo': ['2.8']}


In [16]:
# check for a specific tool
target_tool = "samtools"
print(AddonsManager.check_tool_is_installed(software, target_tool))

samtools installed with version(s):
1.1
